## Esercizio

### Consegna

- Scegliere un verbo transitivo --> **KILL**
- Trovare un corpus con > 1000 frasi in cui comprare un verbo scelto (usare un verbo comune) --> link to resource: https://sentence.yourdictionary.com/kill
- Effettuare parsing e disambiguazione
- Usare i supersensi di wordnet sugli argomenti (subj e obj nel caso di 2 argomenti) del verbo scelto
- Calcolo risultati, frequenza e stampare cluster semantici ottenuti

### Domande per il prof

- Il modo in ci etichettiamo "person" alle parole va bene?
- Il modo in cui disambiguiamo, per quanto poco efficiente (lesk), è ok?
- Va bene il modo in cui rappresentiamo i risultati (cluster semantici)?

### Appunti

Wordnet supersense: Per ogni sinset abbiamo un supersenso associato (esempi su slide)
Utilizzare i supersense dei synset per determinare il semantic type - nltk

I supersense di wordenet non sono il massimo, vediamo alcune alternative:
 * CSI - Ai supersense ci sono associati delle categorie che possono essere utilizzati come supersense

link to wordnet resource: https://www.nltk.org/_modules/nltk/corpus/reader/wordnet.html

*Spunti futuri:*

**Come distinguo i synset?**
--> Funzione lesk (https://www.nltk.org/howto/wsd.html#word-sense-disambiguation)

**Come trovo il supersenso tra due termini?**
--> Funzione Lowest Common Hypernyms (https://www.nltk.org/howto/wordnet_lch.html)

### Esecuzione

In [129]:
from nltk.corpus import wordnet
from spacy.matcher import DependencyMatcher
from nltk.wsd import lesk
import re
import spacy
from pprint import pprint

* person: Usata per trovare il supersenso dei pronomi
* patter: Usato per trovare il soggetto e l'oggetto del verbo kill

In [130]:
person = ["i", "you", "he", "she", "we", "they", "me", "him", "her", "his", "them", "someone", "us", "people", "anyone"] 

pattern1 = [
    {"RIGHT_ID": "attr",
    "RIGHT_ATTRS": {"LEMMA": {"IN": ["kill"]}}
    },
    {"LEFT_ID": "attr",
    "REL_OP": ">",
    "RIGHT_ID": "subj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["nsubj"]}}
    },
    {"LEFT_ID": "attr",
    "REL_OP": ">",
    "RIGHT_ID": "dobj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["dobj"]}}
    }
]

pattern2 = [
    {"RIGHT_ID": "verb",
    "RIGHT_ATTRS": {"LEMMA": {"IN": ["want", "wish"]}}
    },
    {"LEFT_ID": "verb",
    "REL_OP": ">",
    "RIGHT_ID": "subj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["nsubj"]}}
    },
    {"LEFT_ID": "verb",
    "REL_OP": ">",
    "RIGHT_ID": "xcomp",
    "RIGHT_ATTRS": {"DEP": {"IN": ["xcomp"]}}
    },
    {"LEFT_ID": "xcomp",
    "REL_OP": ">",
    "RIGHT_ID": "dobj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["dobj"]}}
    }
]

pattern3 = [
    
]

#### carico spacy e aggiungo il pattern al Matcher

In [131]:
nlp = spacy.load('en_core_web_sm')
matcher = DependencyMatcher(nlp.vocab)
matcher.add("pattern1", [pattern1])
matcher.add("pattern2", [pattern2])

#### Metodi utili per trovare il match e per la wsd

In [132]:
def get_match( text):
    # Find the pattern in the document
    doc = nlp(text)
    matches = matcher(doc)
    for match in matches:
        match_words = sorted(match[1])
        phrase = doc[match_words[0]:match_words[len(match_words)-1]+1]
        subj = phrase[0].text
        dobj = phrase[len(phrase)-1].text
        
        return subj,dobj,phrase[0].tag_,phrase[len(phrase)-1].tag_
    return "","","",""

def word_sense_disambiguation(list_words, word):
    right_synset = lesk(list_words, word)
    return right_synset

In [133]:
def cleaner(text):
    res = text.split('.')
    return res[1]

In [134]:
subj_ss = ""
dobj_ss = ""
struct = {}
tot = 0
c=0
with open ('../sentence_kill.txt', 'r', encoding="utf8") as f:
    for row in f:
        subj_synset, dobj_synset, subj_ss, dobj_ss = None, None, "", ""
        subj, dobj, stag, dtag = get_match(row)
            
        if subj != "" and dobj != "":
            # Cerco il synset del soggetto e dell'oggetto e associo automaticamente il synset "person" se trovo
            # un nome proprio o una sringa presente in person
            
            # Soggetto
            if stag == "NNP" or subj.lower() in person:
                subj_ss = "person"
            else:
                subj_synset = word_sense_disambiguation(re.findall(r'\w+', row), subj)
            # Oggetto
            if dtag == "NNP" or dobj.lower() in person:
                dobj_ss = "person"
            else:
                dobj_synset = word_sense_disambiguation(re.findall(r'\w+', row), dobj)        
            
            # Se subj_synset e' None, significa che abbiamo associato il synset person
            if not subj_synset is None: 
                subj_ss = cleaner(subj_synset.lexname())
            elif subj_ss != "person":
                subj_ss = "unknown"
            
            # Soggetto
            if not dobj_synset is None:
                dobj_ss = cleaner(dobj_synset.lexname())  
            elif dobj_ss != "person":
                dobj_ss = "unknown"

            # Oggetto
            if (subj_ss, dobj_ss) in struct:
                struct[(subj_ss, dobj_ss)] += 1
            else:
                struct[(subj_ss, dobj_ss)] = 1
        else:
            c+=1
            #print(f"phrase {c}: {row}")
for v in struct.values():
    tot += v

for k in struct.keys():
    if k[0] != "unknown" and k[1] != "unknown":
        print(f"{k}: {round(((struct[k]/tot)*100), 2)}%")

('artifact', 'communication'): 0.91%
('person', 'person'): 55.45%
('person', 'group'): 1.36%
('competition', 'artifact'): 0.45%
('person', 'communication'): 2.27%
('food', 'person'): 0.45%
('person', 'animal'): 2.73%
('act', 'person'): 0.91%
('person', 'all'): 4.09%
('person', 'artifact'): 1.82%
('cognition', 'person'): 1.36%
('person', 'cognition'): 0.91%
('body', 'person'): 0.45%
('person', 'stative'): 0.45%
('person', 'act'): 1.36%
('group', 'person'): 2.73%
('communication', 'person'): 0.45%
('contact', 'person'): 0.91%
('person', 'emotion'): 0.91%
('artifact', 'act'): 0.45%
('act', 'cognition'): 0.45%
('all', 'food'): 0.45%
('communication', 'food'): 0.45%
('all', 'person'): 0.45%
('artifact', 'all'): 0.45%
('person', 'food'): 0.45%
('motion', 'person'): 0.45%
('animal', 'quantity'): 0.45%
('social', 'person'): 0.91%
('social', 'state'): 0.45%
('person', 'change'): 0.45%
('group', 'consumption'): 0.45%
('Tops', 'person'): 0.45%
('artifact', 'person'): 0.45%
('food', 'time'): 0.45%